# Datenanalyse

Laden

ansicht

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
csv_path = '../data/roh/airscout_full-spectrum_neustadt_weinstrasse_2025_07190605.csv'
# Finde die erste Zeile ohne '#' als Header
with open(csv_path, encoding='utf-8') as f:
    for i, row in enumerate(f):
        if not row.strip().startswith('#'):
            header_row = i
            break
# Versuche zuerst mit Komma, dann mit Semikolon als Trennzeichen zu laden
try:
    df = pd.read_csv(csv_path, skiprows=header_row-1)
    if df.shape[1] == 1:
        raise ValueError('Nur eine Spalte erkannt, versuche sep=;')
except Exception:
    df = pd.read_csv(csv_path, skiprows=header_row-1, sep=';')
df.reset_index(drop=True, inplace=True)
print('Spaltennamen:', list(df.columns))

Spaltennamen: ['# ===================================']


In [23]:
# Entferne alle Zeit-Zusätze wie ' MESZ', ' UTC', ' utc', 'UTC1', 'UT', 'CET' usw. am Ende der Strings
import re
for col in ['DateTime', 'GPS_DateTime']:
    if col in df.columns:
        df[col] = df[col].str.replace(r'\s*(MESZ|mesz|UTC\d*|utc\d*|UT\d*|ut\d*|CET|cet)$', '', regex=True)

In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import glob
import os

# Lade die CSV-Daten in ein DataFrame
df = pd.read_csv('../data/bearbeitet0/2025_07_25_13_49.csv')

# Berechne die Korrelation zwischen den Sensorwerten und den Umgebungsvariablen
correlation_temp = df[['Temperature_DHT_C', 'MQ2', 'MQ3', 'MQ4', 'MQ5', 'MQ6', 'MQ7', 'MQ8', 'MQ9']].corr()
correlation_humidity = df[['Humidity_RH', 'MQ2', 'MQ3', 'MQ4', 'MQ5', 'MQ6', 'MQ7', 'MQ8', 'MQ9']].corr()

# Visualisiere die Korrelationen
sns.heatmap(correlation_temp, annot=True, cmap='coolwarm')
plt.title('Korrelation zwischen Temperatur und Gassensoren')
plt.show()

sns.heatmap(correlation_humidity, annot=True, cmap='coolwarm')
plt.title('Korrelation zwischen Luftfeuchtigkeit und Gassensoren')

plt.show()




FileNotFoundError: [Errno 2] No such file or directory: '../data/bearbeitet0/2025_07_25_13_49.csv'

In [ ]:
# Korrelationsmatrix auch als Text ausgeben
print('Korrelationsmatrix Temperatur vs. Gassensoren:')
print(correlation_temp.to_string())

print('\nKorrelationsmatrix Luftfeuchtigkeit vs. Gassensoren:')
print(correlation_humidity.to_string())

Korrelationsmatrix Temperatur vs. Gassensoren:
                   Temperature_DHT_C       MQ2       MQ3       MQ4       MQ5       MQ6       MQ7       MQ8       MQ9
Temperature_DHT_C           1.000000 -0.920341 -0.668041 -0.785999 -0.636693 -0.743096 -0.879383 -0.733958 -0.899583
MQ2                        -0.920341  1.000000  0.759854  0.931044  0.848462  0.919900  0.856798  0.916604  0.994072
MQ3                        -0.668041  0.759854  1.000000  0.798840  0.751679  0.767091  0.871672  0.774638  0.772507
MQ4                        -0.785999  0.931044  0.798840  1.000000  0.918577  0.943373  0.781272  0.950026  0.946820
MQ5                        -0.636693  0.848462  0.751679  0.918577  1.000000  0.928832  0.659045  0.934821  0.876245
MQ6                        -0.743096  0.919900  0.767091  0.943373  0.928832  1.000000  0.730798  0.954160  0.937346
MQ7                        -0.879383  0.856798  0.871672  0.781272  0.659045  0.730798  1.000000  0.735637  0.846415
MQ8              

In [ ]:

display(df.head())
df.describe()


,SecSinceMidnight-MS,Temperature_DHT_C,Humidity_RH,Light_Level,Light_Percent,GPS_Lat,GPS_Lon,GPS_Alt,GPS_Speed,GPS_Course,...,MQ6,MQ7,MQ8,MQ9,MQ135,Mic1,Mic2,Radiation_CPS,DateTime,GPS_DateTime
0,50470-346,23.2,63.0,293,71.4,49.3527,8.1644,87.4,3.6,0,...,242,355,330,438,385,2,2,1,2025-07-25 16:01:10,2025-07-25 14:01:11
1,50472-379,23.2,63.0,293,71.4,49.3527,8.1644,85.5,1.9,0,...,241,354,331,438,385,2,3,0,2025-07-25 16:01:12,2025-07-25 14:01:13
2,50474-429,23.2,63.0,291,71.6,49.3527,8.1644,84.2,2.9,0,...,242,353,330,438,384,2,3,1,2025-07-25 16:01:14,2025-07-25 14:01:15
3,50476-466,23.3,63.0,292,71.5,49.3527,8.1644,84.7,3.0,0,...,242,352,330,438,384,3,3,0,2025-07-25 16:01:16,2025-07-25 14:01:17
4,50478-637,23.3,63.0,290,71.7,49.3527,8.1644,83.4,2.3,0,...,243,352,330,437,385,3,2,0,2025-07-25 16:01:18,2025-07-25 14:01:19


,Temperature_DHT_C,Humidity_RH,Light_Level,Light_Percent,GPS_Lat,GPS_Lon,GPS_Alt,GPS_Speed,GPS_Course,GPS_Sats,...,MQ4,MQ5,MQ6,MQ7,MQ8,MQ9,MQ135,Mic1,Mic2,Radiation_CPS
count,1681.000000,1681.000000,1681.000000,1681.000000,1681.000000,1681.000000,1681.00000,1681.000000,1681.000000,1681.000000,...,1681.000000,1681.000000,1681.000000,1681.000000,1681.000000,1681.000000,1681.000000,1681.000000,1681.000000,1681.000000
mean,24.784176,57.092207,326.406901,68.095895,49.352620,8.164538,118.74539,1.065854,4.441999,6.683522,...,387.381321,330.505651,240.529447,345.914337,328.426532,419.946460,371.151695,2.595479,2.613325,0.084474
std,0.469422,1.603328,61.212917,5.986164,0.000074,0.000106,11.33396,0.820025,31.839484,1.435443,...,2.498878,1.990483,2.480974,5.240732,2.662926,9.588348,7.634598,1.238981,1.245626,0.298812
min,23.200000,56.000000,222.000000,56.900000,49.352400,8.163800,79.30000,0.000000,0.000000,4.000000,...,383.000000,326.000000,235.000000,324.000000,324.000000,405.000000,360.000000,1.000000,1.000000,0.000000
25%,24.600000,56.000000,263.000000,63.700000,49.352600,8.164500,112.90000,0.500000,0.000000,6.000000,...,385.000000,329.000000,238.000000,342.000000,326.000000,411.000000,364.000000,2.000000,2.000000,0.000000
50%,25.000000,56.000000,335.000000,67.300000,49.352600,8.164600,119.30000,0.900000,0.000000,7.000000,...,387.000000,330.000000,240.000000,343.000000,328.000000,417.000000,369.000000,2.000000,2.000000,0.000000
75%,25.100000,57.000000,370.000000,74.300000,49.352700,8.164600,124.30000,1.400000,0.000000,8.000000,...,390.000000,332.000000,243.000000,351.000000,331.000000,428.000000,378.000000,3.000000,3.000000,0.000000
max,25.400000,63.000000,441.000000,78.300000,49.353200,8.164800,171.50000,5.700000,353.000000,38.000000,...,393.000000,335.000000,245.000000,374.000000,333.000000,441.000000,387.000000,23.000000,22.000000,3.000000


In [37]:
import glob
import os
import pandas as pd
from sklearn.linear_model import LinearRegression

# Alle CSV-Dateien im gewünschten Ordner finden
csv_folder = '../data/bearbeitet0'
csv_files = glob.glob(os.path.join(csv_folder, '*.csv'))

dfs = []
file_names = []
for f in csv_files:
    df_temp = pd.read_csv(f)
    df_temp['Dateiname'] = os.path.basename(f)
    dfs.append(df_temp)
    file_names.append(os.path.basename(f))

# Alle Daten in einem einzigen DataFrame zusammenführen
df = pd.concat(dfs, ignore_index=True)

# Beispiel für Spaltennamen: ['Temperature_DHT_C', 'Humidity_RH', 'MQ2', 'MQ3', ..., 'MQ9']
# Stellen wir sicher, dass diese Spalten in den CSV-Dateien existieren:
sensor_columns = ['MQ2', 'MQ3', 'MQ4', 'MQ5', 'MQ6', 'MQ7', 'MQ8', 'MQ9']

# Temperatur- und Luftfeuchtigkeitsänderungen berechnen (im Vergleich zu einer Referenzumgebung)
df['Temp_Diff'] = df['Temperature_DHT_C'] - df['Temperature_DHT_C'].mean()
df['Humidity_Diff'] = df['Humidity_RH'] - df['Humidity_RH'].mean()

# Funktion zur Berechnung des Korrekturfaktors für jede Sensorspalte
def calculate_correction_factors(df, sensor_columns):
    rows = []
    for fname in df['Dateiname'].unique():
        df_file = df[df['Dateiname'] == fname]
        for sensor in sensor_columns:
            X = df_file[['Temp_Diff', 'Humidity_Diff']]
            y = df_file[sensor]
            model = LinearRegression()
            model.fit(X, y)
            rows.append({
                'Dateiname': fname,
                'Sensor': sensor,
                'Temp_Factor': model.coef_[0],
                'Humidity_Factor': model.coef_[1]
            })
    return rows

# Berechne die Korrekturfaktoren für jede Datei und jeden Sensor
correction_factors = calculate_correction_factors(df, sensor_columns)

# Kompakte Textausgabe, gruppiert nach Dateiname
from collections import defaultdict
output = defaultdict(list)
for row in correction_factors:
    output[row['Dateiname']].append(f"{row['Sensor']}: Temp_Factor={row['Temp_Factor']:.4f}, Hum_Factor={row['Humidity_Factor']:.4f}")

for fname, lines in output.items():
    print(f"Datei: {fname}")
    for line in lines:
        print(f"  {line}")
    print()

Datei: 2025_07_23_16_15.csv
  MQ2: Temp_Factor=25.5265, Hum_Factor=2.6686
  MQ3: Temp_Factor=13.9322, Hum_Factor=-0.4395
  MQ4: Temp_Factor=18.7606, Hum_Factor=2.5485
  MQ5: Temp_Factor=15.9924, Hum_Factor=2.1883
  MQ6: Temp_Factor=21.0611, Hum_Factor=3.1692
  MQ7: Temp_Factor=5.4300, Hum_Factor=-0.9244
  MQ8: Temp_Factor=16.7054, Hum_Factor=2.2551
  MQ9: Temp_Factor=41.6043, Hum_Factor=0.0574



 # DANGER!
 
 1. lade die csv datei aus dem ordner: E:\dev\projekt_python_venv\airscout-analytics\data\bearbeitet
 2. ersetze alle werte in den Spalten: GPS_Lat,GPS_Lon,GPS_Alt,GPS_Speed,GPS_Course, mit der zahl 8.0
 3. ersetze alle werte in der Spalte GPS_DateTime durch 9999/01/01 00:00:00 UTC

In [40]:



import pandas as pd
import os

# 1. Lade die erste CSV-Datei aus dem Ordner
csv_dir = r'E:/dev/projekt_python_venv/airscout-analytics/data/bearbeitet'
csv_files = [f for f in os.listdir(csv_dir) if f.lower().endswith('.csv')]
if not csv_files:
    raise FileNotFoundError('Keine CSV-Datei im Ordner gefunden!')
csv_path = os.path.join(csv_dir, csv_files[0])
df = pd.read_csv(csv_path)

# 2. Ersetze Werte in den gewünschten Spalten durch 8.0
spalten_ersetzen = ['GPS_Lat','GPS_Lon','GPS_Alt','GPS_Speed','GPS_Course']
for spalte in spalten_ersetzen:
    if spalte in df.columns:
        df[spalte] = 8.0

# 3. Ersetze alle Werte in GPS_DateTime durch den gewünschten String
if 'GPS_DateTime' in df.columns:
    df['GPS_DateTime'] = '2025/01/01 00:00:00 UTC'

# 4. Speichere die veränderte Datei unter dem gleichen Namen
df.to_csv(csv_path, index=False, encoding='utf-8')

# Ausgabe zur Kontrolle
print('Datei geladen und überschrieben:', csv_path)
print(df.head())

Datei geladen und überschrieben: E:/dev/projekt_python_venv/airscout-analytics/data/bearbeitet\alle_rohdaten_zusammen_2025_01010000.csv
  SecSinceMidnight-MS Temperature_DHT_C Humidity_RH  \
0         54311-987;0               0;0     0;21;98   
1        54313-983;36              1;15     0;22;97   
2        54315-063;36              0;15     0;15;98   
3        54318-307;35              9;15     0;16;98   
4        54320-311;35              8;15     0;17;98   

                                         Light_Level Light_Percent  GPS_Lat  \
0  1;XX;XX;XX;XX;XX;0;120;154;361;312;209;244;307...           NaN      8.0   
1  9;XX;XX;XX;XX;XX;0;138;151;370;321;219;249;317...           NaN      8.0   
2  4;XX;XX;XX;XX;XX;0;156;153;375;327;226;256;324...           NaN      8.0   
3  4;XX;XX;XX;XX;XX;0;173;153;380;332;232;262;330...           NaN      8.0   
4  4;XX;XX;XX;XX;XX;0;189;161;384;336;238;273;335...           NaN      8.0   

   GPS_Lon  GPS_Alt  GPS_Speed  GPS_Course  ...  MQ5  MQ6 

# DANGER!!!

In [ ]:
import numpy as np
import pandas as pd

# Korrekturfaktoren aus den Ergebnissen
correction_factors = {
    'MQ2': {'Temp_Factor': 2.8285, 'Hum_Factor': 0.5697},
    'MQ3': {'Temp_Factor': 3.2848, 'Hum_Factor': -0.0621},
    'MQ4': {'Temp_Factor': 3.5289, 'Hum_Factor': 0.5530},
    'MQ5': {'Temp_Factor': 3.4732, 'Hum_Factor': 0.2828},
    'MQ6': {'Temp_Factor': 3.2520, 'Hum_Factor': 0.0183},
    'MQ7': {'Temp_Factor': 0.9904, 'Hum_Factor': -0.0957},
    'MQ8': {'Temp_Factor': 2.9071, 'Hum_Factor': 0.2895},
    'MQ9': {'Temp_Factor': 6.7154, 'Hum_Factor': 0.2320},
}

# 1. Lade die erste CSV-Datei aus dem Ordner
csv_dir = r'E:/dev/projekt_python_venv/airscout-analytics/data/bearbeitet'
csv_files = [f for f in os.listdir(csv_dir) if f.lower().endswith('.csv')]
if not csv_files:
    raise FileNotFoundError('Keine CSV-Datei im Ordner gefunden!')
csv_path = os.path.join(csv_dir, csv_files[0])



# Temperatur- und Feuchtigkeitsdifferenzen berechnen (Differenz vom Mittelwert)
df['Temp_Diff'] = df['Temperature_DHT_C'] - df['Temperature_DHT_C'].mean()
df['Humidity_Diff'] = df['Humidity_RH'] - df['Humidity_RH'].mean()

# Funktion zur Normalisierung der Sensorwerte
def normalize_sensor(df, sensor, temp_factor, hum_factor):
    df_normalized = df.copy()
    
    # Normalisiere Sensorwerte
    df_normalized[sensor] = df[sensor] * (1 + temp_factor * df['Temp_Diff']) * (1 + hum_factor * df['Humidity_Diff'])
    
    return df_normalized

# Anwendung der Normalisierung auf alle Sensoren
for sensor in correction_factors:
    temp_factor = correction_factors[sensor]['Temp_Factor']
    hum_factor = correction_factors[sensor]['Hum_Factor']
    df = normalize_sensor(df, sensor, temp_factor, hum_factor)

# Speichern der normalisierten Daten
df.to_csv('normalized_2025_07_25_13_49.csv', index=False)

# Ausgabe der normalisierten Daten
print(df.head())
